In [56]:
import pandas as pd
import numpy as np 
import seaborn as sns 
from sklearn.cluster import KMeans
from scipy import stats
from sklearn.decomposition import PCA

In [2]:
# use pca?

In [3]:
data_points_1 = pd.read_csv('./data/1_5_CPMcutoff_suffix_1_log_cero_replacement.csv', index_col = 0)
data_points_2 = pd.read_csv('./data/1_5_CPMcutoff_suffix_2_log_cero_replacement.csv', index_col = 0)
data_points_3 = pd.read_csv('./data/1_5_CPMcutoff_suffix_3_log_cero_replacement.csv', index_col = 0)
data_points = pd.merge(left = data_points_1, right = data_points_2, left_index = True, right_index = True)
data_points = pd.merge(left = data_points, right = data_points_3, left_index = True, right_index = True)

In [222]:
class Random_K_Means():
    '''
    K means for random SME initialization
    '''
    def __init__(self, data, cluster_size = 20, pca_dim = 4):
        '''
        data is a pandas dataframe
        '''
        self.data = data
        self.cluster_size = cluster_size
        self.pca_dim = pca_dim
        self.pca()
        
    def k_means_labels(self, pca = False):
        '''
        Returns K means labels with the data and the specified cluster size
        '''
        if pca:
            result = KMeans(n_clusters= 20, tol=1e-8, max_iter= 1000).fit(self.data_pca.values).labels_
        else :
            result = KMeans(n_clusters= 20, tol=1e-8, max_iter= 1000).fit(self.data.values).labels_
        return result
    
    def pca(self):
        '''
        Makes a pca with the data, makes the data a new dataframe with
        the values from the pca
        '''
        pca = PCA(n_components=self.pca_dim, svd_solver='full')
        self.data_pca = pd.DataFrame(pca.fit_transform(self.data.values))
        
    def get_random_starts_limits(
        self,
        labels,
        mean_geom= False,
        std_geom = False
    ):
        data = self.data if not pca else self.data_pca
        means, stds = [],[]
        for label in np.unique(labels):
            sub_data = data[labels == label]
            means.append(
                stats.gmean(sub_data.mean()) if mean_geom else sub_data.mean().mean()
            )
            stds.append(
                stats.gmean(sub_data.std()) if mean_geom else sub_data.std().mean()
            )
        return np.array(means), np.array(stds)
    def get_center_sample_from_data(
        self, 
        pca = False,
        mean_geom = False,
        std_geom = False
    ):
        means, stds = self.get_random_starts_limits(
            foo.k_means_labels(
                    pca = pca,
            ),
            mean_geom= mean_geom,
            std_geom = std_geom
        )
        min_val = means - 2* stds
        for i in range(min_val.size):
            r = np.random.randint(11)
            min_val[i] += r * (stds[i]* 4/10)
        return min_val
        
foo = Random_K_Means(data = data_points)

In [223]:
foo.get_center_sample_from_data()

array([-0.70400123,  0.67857863, -1.62025791,  0.25089805,  2.67915812,
       -2.25444693,  2.57866529, -1.0162908 , -0.65217418, -0.00614044,
        1.66509969,  0.6079784 , -1.53646759,  2.48513418, -3.54366261,
        2.24580868, -1.17518095,  0.27469305,  0.52357198, -1.72169983])

[ 1.11399584  1.33752043  2.42904978 -0.66542412 -0.26152211  1.99651448
  0.07489734 -0.07131973  2.22715781  1.16697071  3.08968594  0.24734934
  3.7732575   1.94268724  3.06899807  0.59220575  1.3884292   2.44327127
  2.35725496 -0.30893816]
5
8
9
10
1
5
9
8
6
3
6
4
6
7
0
10
1
3
3
3
[2.97532007 4.25158774 6.21042241 2.0394897  0.07377332 3.90325439
 2.91050297 3.10006555 4.47411316 2.36275656 5.58612033 1.60563409
 6.76633725 4.60819808 3.06899807 4.36395605 1.7564205  3.53093189
 3.45713699 0.48474543]


In [211]:
idx = 0
means[idx], stds[idx], means[idx]+2*stds[idx]

(2.9753200700812967, 0.9306621133800794, 4.836644296841456)

In [14]:
%%time
kmeans = KMeans(n_clusters= 20, tol=1e-8, max_iter= 1000).fit(data_points.values)

CPU times: user 2.06 s, sys: 2.8 ms, total: 2.06 s
Wall time: 2.41 s


In [18]:
labels = pd.Series(kmeans.labels_)

In [73]:
data_points[ kmeans.labels_ == 6 ].mean().mean()

4.059673003715914

In [40]:
stats.gmean(data_points[ kmeans.labels_ == 6 ].mean())

3.9568046329150937

In [46]:
stats.gmean(data_points[ kmeans.labels_ == 6 ].std())

0.922530908616717